In [ ]:
#install: download disentanglement_lib and follow instructions to download datasets
!git clone https://github.com/google-research/disentanglement_lib.git
!mv disentanglement_lib disentanglement_library
!cp -r disentanglement_lib_patch/* disentanglement_library/disentanglement_lib/

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./disentanglement_library/')

import os
os.environ["DISENTANGLEMENT_LIB_DATA"] = "./disentanglement_library/data/"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from argparse import ArgumentParser
from multiprocessing import Pool

import torch
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from models.model import Model
from models.conv64 import SimpleConv64,SimpleConv64D

parser = ArgumentParser(add_help=False)
parser = Trainer.add_argparse_args(parser)
parser.set_defaults(max_epochs=-1)
parser.set_defaults(check_val_every_n_epoch=1)
parser = Model.add_model_specific_args(parser)
args = parser.parse_args(['--n_dataset','2',
                          '--k','-1',
                          '--latentdim','10',
                          '--n_latent_factors', '10',
                          '--learning_rate','5e-4',
                          '--max_epochs','30',
                          '--gpus','1',
                          '--progress_bar_refresh_rate','10',
                          '--limit_val_batches','1',
                          '--check_val_every_n_epoch','1',
                          '--log_every_n_steps','160',
                           ])

def train(n_dataset):
    args.n_dataset = n_dataset
    
    ch=3
    if n_dataset==0 or n_dataset==5:
        ch=1
    
    model = Model(hparams=args, encoder=SimpleConv64(args.latentdim,ch,64,batch_size=args.batch_size,with_bn=False),
                                decoder=SimpleConv64D(args.latentdim,ch,64,with_bn=False))
    # basic trainer
    checkpoint_callback = ModelCheckpoint(
        save_last=True,
        save_top_k=1,
        verbose=True,
        monitor='val_loss',
        mode='min'
    )

    run_name = 'TEST_vark_data_%d' % n_dataset
    wandb_logger = WandbLogger(project="lat_geo",log_model=True,name=run_name,reinit=True)
    trainer = Trainer.from_argparse_args(args, logger=wandb_logger,checkpoint_callback=checkpoint_callback) #without 16 bit precision
    trainer.fit(model)


train(2)